In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Read data
sem_data = pd.read_csv("beetleTrainingData.csv")

In [7]:
# Data stat
total_data = len(sem_data.index)
total_features = len(sem_data.columns) -1 #last column is the class labels
total_categories = 4

In [8]:
#Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

In [13]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, total_features]) # sentence data with total vocabulary size
y = tf.placeholder(tf.float32, [None, 1]) 

In [14]:
# Set model weights
W = tf.Variable(tf.zeros([total_features, total_categories]))
b = tf.Variable(tf.zeros([total_categories]))

In [15]:
# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [16]:
# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [18]:
#Contruct data and separate into training and testing stes
X = sem_data.iloc[:, 0:total_features]
y_class = np.ravel(sem_data.accuracy)
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.33,
                                                    random_state=42)

In [19]:
# Define the scaler
scaler = StandardScaler().fit(X_train)
# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [20]:
# Initializing the variables
init = tf.global_variables_initializer()

In [23]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_data / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch = i * batch_size
            batch_xs, batch_ys = X_train[batch + i:batch + batch_size + i], y_train[
                                                                      batch + i:batch
                                                                                + batch_size + i]
            if len(batch_ys) ==0:
                continue
            # Transform to scaler    
            scalar = StandardScaler().fit(batch_xs)
            batch_xs = scalar.transform(batch_xs)
            batch_ys = batch_ys.reshape(len(batch_ys), 1)
            scalar = StandardScaler().fit(batch_ys)
            batch_ys= scalar.transform(batch_ys)
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    y_test = y_test.reshape(len(y_test), 1)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x:X_test, y:y_test}))

C:\Users\sjp967\AppData\Local\Continuum\Miniconda3\envs\python3.5\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Epoch: 0001 cost= -0.000000008
Epoch: 0002 cost= -0.000000008
Epoch: 0003 cost= -0.000000008
Epoch: 0004 cost= -0.000000008
Epoch: 0005 cost= -0.000000008
Epoch: 0006 cost= -0.000000008
Epoch: 0007 cost= -0.000000008
Epoch: 0008 cost= -0.000000008
Epoch: 0009 cost= -0.000000008
Epoch: 0010 cost= -0.000000008
Epoch: 0011 cost= -0.000000008
Epoch: 0012 cost= -0.000000008
Epoch: 0013 cost= -0.000000008
Epoch: 0014 cost= -0.000000008
Epoch: 0015 cost= -0.000000008
Epoch: 0016 cost= -0.000000008
Epoch: 0017 cost= -0.000000008
Epoch: 0018 cost= -0.000000008
Epoch: 0019 cost= -0.000000008
Epoch: 0020 cost= -0.000000008
Epoch: 0021 cost= -0.000000008
Epoch: 0022 cost= -0.000000008
Epoch: 0023 cost= -0.000000008
Epoch: 0024 cost= -0.000000008
Epoch: 0025 cost= -0.000000008
Optimization Finished!
Accuracy: 1.0
